In [45]:
from numpy import pi, linspace, diff, mean, min, max, arctan2
from skyfield.api import load
from skyfield.framelib import ecliptic_J2000_frame
from skyfield.searchlib import find_discrete
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [46]:
output_notebook()

Loading BokehJS ...

In [47]:
ts = load.timescale()
eph_s_name = 'de421.bsp'
# eph_s_name = 'de422.bsp'
t_start = ts.tt(1950, 6, 21, 0, 0, 0)
eph_s = load(eph_s_name)
times = ts.tt_jd(linspace(t_start.tt, ts.tt(1952, 6, 21, 0, 0, 0).tt, 1000))
moon = eph_s["moon"]
earth = eph_s["earth"]
sun = eph_s["sun"]

## Verify sidereal and synodic periods of the Moon

In [48]:
def f_x_zero(target):
    def zero(t):
        x, _, _ = (target.at(t) - earth.at(t)).frame_xyz(ecliptic_J2000_frame).au
        return x > 0
    zero.step_days = 10.0
    return zero

In [49]:
x_0_cross = find_discrete(t_start, t_start + 365*50, f_x_zero(moon))
x_0_node = x_0_cross[0][x_0_cross[1] == 1]
mean(diff(x_0_node)), min(diff(x_0_node)), max(diff(x_0_node)), len(diff(x_0_node))

(np.float64(27.321923193710994),
 np.float64(27.222805836237967),
 np.float64(27.47844494925812),
 667)

In [50]:
def f_cross_sun():
    def zero(t):
        _, lon_moon, _ = (moon.at(t) - earth.at(t)).frame_latlon(ecliptic_J2000_frame)
        _, lon_sun, _ = (sun.at(t) - earth.at(t)).frame_latlon(ecliptic_J2000_frame)
        return ((lon_moon.radians - lon_sun.radians) / pi % 2).astype('int') > 0
    zero.step_days = 10.0
    return zero

In [51]:
sun_cross = find_discrete(t_start, t_start + 365*50, f_cross_sun())
synodic_man = diff(sun_cross[0][sun_cross[1] == 1])
mean(synodic_man), min(synodic_man), max(synodic_man), len(synodic_man)

(np.float64(29.53076830931835),
 np.float64(29.27383548161015),
 np.float64(29.82000264292583),
 617)

## Calculate the moon node times and positions

In [52]:
def f_nodes():
    def zero(t):
        _, _, z = (moon.at(t) - earth.at(t)).frame_xyz(ecliptic_J2000_frame).au
        return z > 0
    zero.step_days = 10.0
    return zero

In [53]:
ecliptic_cross = find_discrete(t_start, t_start + 365*100, f_nodes())
t_node = ecliptic_cross[0][ecliptic_cross[1] == 1]
pos_node = (moon.at(t_node) - earth.at(t_node)).frame_xyz(ecliptic_J2000_frame).au
node_lon = arctan2(pos_node[1], pos_node[0])

In [54]:
colors = [f"#{255:02x}{int(255 * (t_node.tt[-1] - t) / (t_node.tt[-1] - t_node.tt[0])):02x}{255:02x}" for t in t_node.tt]
fig = figure(title="Moon nodes", x_axis_label="x(AU)", y_axis_label="y(AU)", width=1100)
fig.scatter(pos_node[0], pos_node[1], fill_color=colors, line_color="red")
fig.line(pos_node[0], pos_node[1], line_color="red")
show(fig)

In [55]:
colors = [f"#{255:02x}{int(255 * (t_node.tt[-1] - t) / (t_node.tt[-1] - t_node.tt[0])):02x}{255:02x}" for t in t_node.tt]
fig = figure(title="Moon ascending nodes", x_axis_label="t_Julian", y_axis_label="angle from x-axis (hours)", width=1100)
fig.scatter(t_node.tt, node_lon*180/pi , fill_color=colors, line_color="red")
show(fig)

In [56]:
t_node_zero = []
for i in range(1, len(node_lon)):
    if node_lon[i-1] > 0 and node_lon[i] <= 0:
        t_node_zero += [ts.tt_jd((t_node.tt[i-1] + t_node.tt[i])/2.0)]
diff(t_node_zero)/365.25

array([np.float64(18.70069792792507), np.float64(18.62562938147364),
       np.float64(18.551667854849), np.float64(18.5518071145727),
       np.float64(18.551766706378157)], dtype=object)

The period of ascending node is close to the official period of 18.61 years:

In [57]:
mean(diff(t_node_zero)/365.25)

np.float64(18.596313797039713)